# GRADED ASSIGNMENT 2

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Week2_GA_dataset.csv to Week2_GA_dataset.csv


In [ ]:
import pandas as pd

df = pd.read_csv("Week2_GA_dataset.csv")
df.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [ ]:
!pip install scikit-learn==1.2.2 numpy==1.23.5

In [ ]:
import numpy as np

In [ ]:
X = df.drop("Target", axis=1)
y = df["Target"]
X.shape

(748, 5)

In [ ]:
row_sum = (X == "?").sum()
row_sum

,0
V1,5
V2,5
V3,0
V4,0
V5,0


In [ ]:
X.replace("?", np.nan, inplace=True)
(X== "?").sum()


,0
V1,0
V2,0
V3,0
V4,0
V5,0


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_cols_imp_scale = ["V1", "V2"]
num_cols_all = ["V1", "V2", "V3", "V4"]
cat_cols = ["V5"]

col_trans = ColumnTransformer(
    transformers=[
        ("imputer", SimpleImputer(strategy="mean"), num_cols_imp_scale)
    ],
    remainder = "passthrough")


col_trans_2 = ColumnTransformer(
    transformers=[
        ("encoder", OrdinalEncoder(), cat_cols),
        ("scaler", StandardScaler(), num_cols_all)
    ],
    remainder="drop")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold

numerical_cols_impute_only = ['V1', 'V2']
numerical_cols_all = ['V1', 'V2', 'V3', 'V4']
categorical_cols = ['V5']

initial_preprocessor = ColumnTransformer(
    transformers=[
        ('impute_numerical_subset', SimpleImputer(strategy='mean'), numerical_cols_impute_only),
        ('passthrough_numerical_others', 'passthrough', ['V3', 'V4']),
        ('encode_categorical', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
    ],
    remainder='drop'
)

X_transformed_initial = initial_preprocessor.fit_transform(X)

print("\nShape after initial_preprocessor:", X_transformed_initial.shape)
print("Sample after initial_preprocessor (first 3 rows):\n", X_transformed_initial[:3])
numerical_part_for_scaling = X_transformed_initial[:, :4]

scaler = StandardScaler()

numerical_part_scaled = scaler.fit_transform(numerical_part_for_scaling)

encoded_categorical_part = X_transformed_initial[:, 4:]

X_scaled_and_encoded = np.hstack((numerical_part_scaled, encoded_categorical_part))

print("\nShape after scaling and re-stacking:", X_scaled_and_encoded.shape)
print("Sample after scaling and re-stacking (first 3 rows):\n", X_scaled_and_encoded[:3])


variance_selector = VarianceThreshold(threshold=0.1)

X_final = variance_selector.fit_transform(X_scaled_and_encoded)

print(f"\nShape after VarianceThreshold: {X_final.shape}")
print("Sample after VarianceThreshold (first 3 rows):\n", X_final[:3])




Shape after initial_preprocessor: (748, 5)
Sample after initial_preprocessor (first 3 rows):
 [[2.00000000e+00 5.00000000e+01 1.25000000e+04 9.80000000e+01
  0.00000000e+00]
 [0.00000000e+00 1.30000000e+01 3.25000000e+03 2.80000000e+01
  0.00000000e+00]
 [9.56258412e+00 5.46433378e+00 4.00000000e+03 3.50000000e+01
  0.00000000e+00]]

Shape after scaling and re-stacking: (748, 5)
Sample after scaling and re-stacking (first 3 rows):
 [[-9.38169390e-01  7.70986653e+00  7.62334626e+00  2.61563344e+00
   0.00000000e+00]
 [-1.18627754e+00  1.30454949e+00  1.28273826e+00 -2.57880900e-01
   0.00000000e+00]
 [ 0.00000000e+00 -1.53758496e-16  1.79684161e+00  2.94705348e-02
   0.00000000e+00]]

Shape after VarianceThreshold: (748, 4)
Sample after VarianceThreshold (first 3 rows):
 [[-9.38169390e-01  7.70986653e+00  7.62334626e+00  2.61563344e+00]
 [-1.18627754e+00  1.30454949e+00  1.28273826e+00 -2.57880900e-01]
 [ 0.00000000e+00 -1.53758496e-16  1.79684161e+00  2.94705348e-02]]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

estimator = LogisticRegression()

rfe_selector = RFE(estimator=estimator, n_features_to_select=2)
rfe_selector.fit(X_final, y)
X_rfe_selected = rfe_selector.transform(X_final)

rfe_selector.ranking_

array([1, 3, 1, 2])

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs_selector = SequentialFeatureSelector(
    estimator=estimator,
    n_features_to_select=2,
    direction='forward',
    cv=None, # No cross-validation needed for this specific problem (full training set)
    scoring='accuracy', # A common metric for classification
    n_jobs=-1 # Use all available cores for faster computation
)

# 3. Fit SFS to your preprocessed data (X_final) and the target (y)
sfs_selector.fit(X_final, y)

# 4. Get the boolean mask of selected features
selected_features_mask_sfs = sfs_selector.get_support()
print("\nBoolean mask of features selected by SFS:", selected_features_mask_sfs)

# 5. Get the indices of the selected features in X_final
sfs_selected_feature_indices = np.where(selected_features_mask_sfs)[0]
print("Indices of the 2 most important features computed by SFS (forward):", sfs_selected_feature_indices)

# --- Transform the data to keep only the selected features ---
X_sfs_selected = sfs_selector.transform(X_final)

print("\nShape after SFS feature selection (should be 2 columns):", X_sfs_selected.shape)
print("Sample after SFS feature selection (first 3 rows):\n", X_sfs_selected[:3])


Boolean mask of features selected by SFS: [False  True False  True]
Indices of the 2 most important features computed by SFS (forward): [1 3]

Shape after SFS feature selection (should be 2 columns): (748, 2)
Sample after SFS feature selection (first 3 rows):
 [[ 7.70986653e+00  2.61563344e+00]
 [ 1.30454949e+00 -2.57880900e-01]
 [-1.53758496e-16  2.94705348e-02]]


In [ ]:

from sklearn.feature_selection import SequentialFeatureSelector

sfs_selector = SequentialFeatureSelector(
    estimator=estimator,
    n_features_to_select=2,
    direction='backward',
    cv=None, # No cross-validation needed for this specific problem (full training set)
    scoring='accuracy', # A common metric for classification
    n_jobs=-1 # Use all available cores for faster computation
)

# 3. Fit SFS to your preprocessed data (X_final) and the target (y)
sfs_selector.fit(X_final, y)

# 4. Get the boolean mask of selected features
selected_features_mask_sfs = sfs_selector.get_support()
print("\nBoolean mask of features selected by SFS:", selected_features_mask_sfs)

# 5. Get the indices of the selected features in X_final
sfs_selected_feature_indices = np.where(selected_features_mask_sfs)[0]
print("Indices of the 2 most important features computed by SFS (forward):", sfs_selected_feature_indices)

# --- Transform the data to keep only the selected features ---
X_sfs_selected = sfs_selector.transform(X_final)

print("\nShape after SFS feature selection (should be 2 columns):", X_sfs_selected.shape)
print("Sample after SFS feature selection (first 3 rows):\n", X_sfs_selected[:3])


Boolean mask of features selected by SFS: [False False  True  True]
Indices of the 2 most important features computed by SFS (forward): [2 3]

Shape after SFS feature selection (should be 2 columns): (748, 2)
Sample after SFS feature selection (first 3 rows):
 [[ 7.62334626  2.61563344]
 [ 1.28273826 -0.2578809 ]
 [ 1.79684161  0.02947053]]
